In [1]:
import streettransformer
import st_preprocessing

from streettransformer.config.constants import DATA_PATH, UNIVERSES_PATH, YEARS

import pandas as pd
import geopandas as gpd
from pathlib import Path

#UNIVERSE_NAME = 'caprecon_plus_control'
UNIVERSE_NAME = 'caprecon_control5k'
universe_path = UNIVERSES_PATH / UNIVERSE_NAME

INFILE = DATA_PATH / 'runtime' / 'results' / UNIVERSE_NAME / 'image_describer_trial1_normal.txt'

locations_gdf = gpd.read_feather(universe_path / 'locations.feather')
locations_gdf = locations_gdf.to_crs('4326')


FileNotFoundError: [Errno 2] Failed to open local file '/Users/jon/Documents/Employment/2025 VIDA Lab/streetTransformer/data/runtime/universes/caprecon_control5k/locations.feather'. Detail: [errno 2] No such file or directory

In [3]:



# with INFILE.open('r') as f:
#     s = f.readlines()


# s

#pd.read_json(INFILE, lines=True)['response']
import json


with INFILE.open('r') as f:
    s = f.readlines()
    
results = [json.loads(l) for l in s] 
responses = [json.loads(r['response']) for r in results if 'response' in r]
responses_df = pd.DataFrame(responses)
responses_df['changes_detected'] = responses_df['significant_changes'].fillna(responses_df["significant_changes_detected"])
responses_df = responses_df.drop(['significant_changes', 'significant_changes_detected'], axis=1)
responses_df[responses_df['changed_features'].apply(len) > 0] # 13/ 96
responses_df[responses_df['changed_features'].apply(len) == 0] # 13

responses_df['changed_features'].value_counts() # ok they are all the same -- that's maybe a problem.




/var/folders/t8/dm9l8xy95mv0d_75b2m8r5nw0000gn/T/ipykernel_84230/520134044.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  responses_df['changes_detected'] = responses_df['significant_changes'].fillna(responses_df["significant_changes_detected"])


changed_features
[]                                                  83
[Bike Lanes, Sidewalks, Traffic Reconfiguration]    13
Name: count, dtype: int64

In [ ]:
# get all locations for controls
# locations_caprecon_gdf = gpd.read_feather(UNIVERSES_PATH / 'caprecon3' / 'locations.feather')
# caprecon_location_ids = locations_caprecon_gdf['location_id']

# locations_3k_gdf = gpd.read_parquet(UNIVERSES_PATH / 'control3k' / 'locations.parquet')
# locations_3k_gdf

# locations_original_gdf = gpd.read_feather(UNIVERSES_PATH / 'caprecon_plus_control' / 'locations.feather')
# locations_original_gdf

# locations_combined_gdf = (
#     pd.concat([
#         locations_original_gdf,
#         locations_3k_gdf.drop('OBJECTID', axis=1).reset_index(drop=True),
#         ], keys=['original', 'control3k']
#     )
#     .reset_index()
#     .drop('level_1', axis=1)
#     .rename({'level_0': 'source'}, axis=1)
#     [['location_id','crossstreets', 'geometry','source']]
# )

# import numpy as np
# treatment_mask = locations_combined_gdf['location_id'].isin(caprecon_location_ids)
# new_source = np.where(treatment_mask, 'caprecon', locations_combined_gdf['source'])
# locations_combined_gdf['source'] = new_source
# locations_combined_gdf['source'] = np.where(locations_combined_gdf['source'] == 'original', 'control2k', locations_combined_gdf['source'])

# locations_combined_gdf['tc_var'] = np.where(locations_combined_gdf['source'] == 'caprecon', 'treat', 'control')
# locations_combined_gdf.value_counts(subset=['source','tc_var']).reset_index()

# # NEW_UNIVERSE_NAME = 'caprecon_control5k'
# # outfile = UNIVERSES_PATH / NEW_UNIVERSE_NAME / 'locations' / 'locations_raw.parquet'
# # outfile.parent.mkdir(exist_ok=True, parents=True)
# # locations_combined_gdf
# # #locations_combined_gdf.to_parquet(outfile)

,source,tc_var,count
0,caprecon,treat,5342
1,control3k,control,3000
2,control2k,control,2000


In [ ]:
import pandas as pd
import geopandas as gpd
locations_combined_gdf = gpd.read_parquet(UNIVERSES_PATH / 'caprecon_control5k' / 'locations' / 'locations_raw.parquet')
locations_combined_gdf.to_crs('4326')

import duckdb
DB_PATH = '../dot_interventions2.duckdb'
con = duckdb.connect(DB_PATH)
con.query('select distinct(doc_id, intervention, page_number) from interventions;') # 3328
#con.query('select distinct(doc_id, intervention, page_number, ) from interventions where intervention != \'bronx\';') # 3328
#con.query('select * from interventions where intervention != \'bronx\' limit 1;') # 3328
con.query('select distinct(doc_id, intervention, page_number, sentence) from interventions where intervention != \'bronx\';') # 3328
df =  con.query('select distinct proj_id, doc_id, intervention_category, intervention, page_number, sentence from interventions;').df()

df[['proj_id', 'intervention_category']].value_counts().unstack() # Only 580 project_ids interesting!

df[['proj_id', 'intervention_category']].value_counts().unstack().sum()
# df[['proj_id', 'doc_subid']] = df['doc_id'].str.split('--', expand=True)
# df[['proj_id', 'intervention']].value_counts()



df[df['intervention'].str.contains('plaza')]

,proj_id,doc_id,intervention_category,intervention,page_number,sentence
52,430,430--1,Pedestrian-related,pedestrian plaza,3,nyc.gov/dot \nWESTCHESTER \nSQUARE BID \nThe W...
372,841,841--0,Pedestrian-related,pedestrian plaza,26,26\nWYCKOFF PEDESTRIAN PLAZA\nReconstruction o...
374,849,849--4,Pedestrian-related,pedestrian plaza,12,IN PERSON WORKSHOP FEEDBACK\n23%\n36%\n49%\n29...
391,866,866--1,Pedestrian-related,pedestrian plaza,1,Avenue B Open Street\nInitial Concepts\nGoals:...
431,104,104--0,Pedestrian-related,pedestrian plaza,11,At West 14th Street \nChelsea (West 14th Stree...
821,916,916--1,Pedestrian-related,pedestrian plaza,5,Downtown Far Rockaway Urban Design \nand Stree...
822,916,916--1,Pedestrian-related,pedestrian plaza,20,Mott Ave\nCentral Ave\nBeach Channel Dr\nLoret...
833,151,151--1,Pedestrian-related,pedestrian plaza,55,Options for the community: School Safety traff...
912,662,662--2,Pedestrian-related,pedestrian plaza,1,Gates Pedestrian Plaza\nMyrtle Avenue \nClasso...
984,841,841--0,Pedestrian-related,pedestrian plaza,2,Reconstruction of Wyckoff Avenue | HWK876\nBor...


In [7]:
df

,proj_id,doc_id,intervention_category,intervention,page_number,sentence
0,919,919--8,Pedestrian-related,median,8,8\nDemographics\nCensus Tracts near the study ...
1,927,927--0,Pedestrian-related,refuge island,16,nyc.gov/visionzero\nPlanned Improvements\nBack...
2,930,930--2,Pedestrian-related,median,17,nyc.gov/visionzero\n17\nDetails of Proposal\nH...
3,931,931--2,traffic and signal control,traffic calming,14,"nyc.gov/dot\n14\n39th Ave in Sunnyside, Queens..."
4,933,933--0,curb management,loading zone,7,nyc.gov/dot\n7\nExisting Conditions\nCongestio...
...,...,...,...,...,...,...
3323,17,17--0,traffic and signal control,one-way conversion,13,13\nProject Highlights\n• Safety Improvements ...
3324,17,17--0,traffic and signal control,road diet,13,13\nProject Highlights\n• Safety Improvements ...
3325,26,26--0,Pedestrian-related,median,3,Hanson Place/Third Avenue Safety \nImprovement...
3326,48,48--0,Pedestrian-related,refuge island,5,Bushwick Avenue at Metropolitan Avenue:\nPropo...


In [ ]:
# import shutil
#locations_combined_gdf['source'].value_counts()
#locations_combined_gdf['tc_var'].value_counts()

locations_combined_gdf.to_crs('4326')


,location_id,crossstreets,geometry,source,tc_var
0,2196,"[PHILIP AVE, AMBOY RD, ALBEE AVE]",POINT (-74.18196 40.53555),caprecon,treat
1,2198,"[AMBOY RD, ANNADALE RD]",POINT (-74.18083 40.53592),caprecon,treat
2,2258,"[ANNADALE RD, POILLON AVE]",POINT (-74.17965 40.53696),caprecon,treat
3,2259,"[AMBOY RD, COTTONTAIL CT]",POINT (-74.17989 40.53572),caprecon,treat
4,2267,"[FURMAN ST, ANNADALE RD, ELY CT]",POINT (-74.17912 40.53749),caprecon,treat
...,...,...,...,...,...
10337,73038,"[VETERANS AVE, ALLEY]",POINT (-73.91078 40.61994),control3k,control
10338,97372,[CROSS ISLAND PKWY],POINT (-73.81899 40.78914),control3k,control
10339,9014464,[PIER],POINT (-73.80467 40.79617),control3k,control
10340,61030,"[GRAND CENTRAL PKWY, GRAND CENTRAL PKWY ET 21 ...",POINT (-73.7567 40.72623),control3k,control
